In [23]:
!pip install openai elasticsearch<=9.0 docx

/bin/bash: =9.0: No such file or directory


In [1]:
!wget "https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py"

--2025-06-18 00:06:37--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-18 00:06:37 (58.7 MB/s) - ‘minsearch.py.2’ saved [4073/4073]



In [2]:
import pandas as pd 
import numpy as np
import openai
from openai import OpenAI
import os

import minsearch

import io
import requests
import json 
#import docx

/workspaces/llm-zoomcamp/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [ ]:
os.environ["OPENAI_API_KEY"] = ""

#### 1. Load the Google Doc document and parse into JSON format 

In [4]:
def clean_line(line):
    line = line.strip()
    line = line.strip('\uFEFF')
    return line

def read_faq(file_id):
    url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'
    
    response = requests.get(url)
    response.raise_for_status()
    
    with io.BytesIO(response.content) as f_in:
        doc = docx.Document(f_in)

    questions = []

    question_heading_style = 'heading 2'
    section_heading_style = 'heading 1'
    
    heading_id = ''
    section_title = ''
    question_title = ''
    answer_text_so_far = ''
     
    for p in doc.paragraphs:
        style = p.style.name.lower()
        p_text = clean_line(p.text)
    
        if len(p_text) == 0:
            continue
    
        if style == section_heading_style:
            section_title = p_text
            continue
    
        if style == question_heading_style:
            answer_text_so_far = answer_text_so_far.strip()
            if answer_text_so_far != '' and section_title != '' and question_title != '':
                questions.append({
                    'text': answer_text_so_far,
                    'section': section_title,
                    'question': question_title,
                })
                answer_text_so_far = ''
    
            question_title = p_text
            continue
        
        answer_text_so_far += '\n' + p_text
    
    answer_text_so_far = answer_text_so_far.strip()
    if answer_text_so_far != '' and section_title != '' and question_title != '':
        questions.append({
            'text': answer_text_so_far,
            'section': section_title,
            'question': question_title,
        })

    return questions



In [8]:
# Sample Code to read Google Doc
faq_documents = {
    'data-engineering-zoomcamp': '19bnYs80DwuUimHM65UV3sylsCn2j1vziPOwzBwQrebw',
    'machine-learning-zoomcamp': '1LpPanc33QJJ6BSsyxVg-pWNMplal84TdZtq10naIhD8',
    'mlops-zoomcamp': '12TlBfhIiKtyBv8RnsoJR6F72bkPDGEvPOItJIxaEzE0',
}

documents = []

for course, file_id in faq_documents.items():
    print(course)
    course_documents = read_faq(file_id)
    documents.append({'course': course, 'documents': course_documents})

with open('documents.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

data-engineering-zoomcamp


KeyboardInterrupt: 

#### 2. Read the Saved Document 
Read the JSON format doc of the course FAQ

In [5]:
!wget "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"

--2025-06-18 00:07:15--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.2’

documents.json.2    100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2025-06-18 00:07:16 (51.8 MB/s) - ‘documents.json.2’ saved [658332/658332]



In [6]:
# Read the file into workspace
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

docs = []

for doc_dict in docs_raw:
    for doc in doc_dict['documents']:
        doc['course'] = doc_dict['course']
        docs.append(doc)

In [7]:
docs[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

#### 3. Index the Documents

In [8]:
# Specify the fields to be indexed 
index = minsearch.Index(text_fields=['question','text','section'],
                keyword_fields=['course'])

index.fit(docs)

#### 4. Sample Query Search

In [9]:
# Retrive the most relavent doc for the query 
def search(query, index, filter_dict):
    boost = {'question': 3.0, 'section': 0.5} # Specify the relative importance of the search on topics


    results = index.search(
        query=query,
        filter_dict = filter_dict,
        boost_dict=boost,
        num_results=5
    )

    # Put all the retrived document text into a signle content
    context = ""
    for ans in results:
        context += ans['text'] + "\n"

    return context

#### 5. Response Generation with Retrival Knowledge

In [48]:
# Make the prompt to embed both the user query and retrived content and generate the answer
def response(query, context):

    # 1. Make the prompt
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}
CONTEXT: {context}f
""".strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()

    # 2. Get the answer using LLM
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [11]:
# Sample Answer
query = "Can I use my own laptop for this course?"

context = search(query, index, filter_dict={'course': 'data-engineering-zoomcamp'})
answer = response(query, context)

answer 

'Yes, you can use your own laptop for this course. You can set it up on your laptop or PC if you prefer to work locally.'

#### 6. Integrate the Components into a Single Function

In [12]:
def rag(user_query, index_repo, filter_dict={'course': 'data-engineering-zoomcamp'}):

    context = search(query = user_query, index = index_repo, filter_dict=filter_dict)
    answer = response(query = user_query, context = context)
    
    return answer  

In [13]:
ans = rag(
    user_query= "How can I set up my environment for this course?",
    index_repo = index
)

print(ans)

To set up your environment for the course, you have a couple of options depending on your preference:

1. **Local Machine Setup:**
   - If you prefer to work locally on your laptop or PC, begin by reviewing the Week 1 Introduction to Docker. This will guide you in setting up your environment locally.
   - Clone the course repository to your local machine using the instructions from the video "Git for Everybody: How to Clone a Repository from GitHub" to easily access course materials and make contributions.
   - Remember to use a `.gitignore` file to exclude large databases, .csv, .gz files, and sensitive information like passwords or keys when managing your repositories.

2. **Virtual Machine Setup:**
   - For a more flexible environment that can be accessed from multiple locations, consider setting up a virtual machine.
   - Utilize GitHub Codespaces or cloud-based environments like using a cloudV Mcodespace for setting up your environment virtually.

Regardless of your choice, if you

#### 7. Replace the Search Engine with ElasticSearch

In [39]:
from elasticsearch8 import Elasticsearch
import elasticsearch8

In [40]:
print(elasticsearch8.__version__)

(8, 18, 1)


In [41]:
# Build the ElasticSearch Host and Index Storage for Loaded Documents
es_client = Elasticsearch(['http://localhost:9200'])

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-homework"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-homework'})

In [42]:
# Indexing the documents
import requests 
from tqdm.auto import tqdm

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:03<00:00, 309.51it/s]


##### define the elasticsearch function

In [43]:
def elastic_search(query, filter_dict, top_n):
    search_query = {
        
        "size": top_n,

        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["questionr^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": filter_dict
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [49]:
# update the rag function
def rag(query, filter_dict, top_n):

    context = elastic_search(query, filter_dict, top_n)
    answer = response(query, context = context)
    
    return context, answer

Try sample Q&A

In [50]:
query = "How do copy a file to a Docker container?"

context, answer = rag(query, filter_dict = "machine-learning-zoomcamp", top_n = 3)

print(answer)

To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax for copying a file or directory from your local machine into a running Docker container is as follows:

```bash
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```

Make sure to replace `/path/to/local/file_or_directory` with the path to the file or directory you want to copy, `container_id` with the actual ID of your running Docker container, and `/path/in/container` with the destination path inside the container.
